In [ ]:
# Instalar las dependencias necesarias
!pip install ultralytics  # Instala la biblioteca Ultralytics para YOLOv8
!pip install torch torchvision  # Instala PyTorch si no está instalado

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 911.6/911.6 kB 46.3 MB/s eta 0:00:00


In [ ]:
from google.colab import files

# Cargar el archivo ZIP
uploaded = files.upload()

Saving images_train.zip to images_train.zip
Saving images_val.zip to images_val.zip
Saving labels_test.zip to labels_test.zip
Saving labels_train.zip to labels_train.zip
Saving labels_val.zip to labels_val.zip
Saving images_test.zip to images_test.zip


In [ ]:
import zipfile
import os

# Ruta base para los archivos en /content
base_dir = '/content'

# Definir la estructura deseada
structure = {
    'train': {
        'images': 'images_train.zip',
        'labels': 'labels_train.zip'
    },
    'val': {
        'images': 'images_val.zip',
        'labels': 'labels_val.zip'
    },
    'test': {
        'images': 'images_test.zip',
        'labels': 'labels_test.zip'
    }
}

# Procesar y descomprimir los archivos según la estructura
for folder, files in structure.items():
    # Crear carpeta principal si no existe
    folder_path = os.path.join(base_dir, folder)
    os.makedirs(folder_path, exist_ok=True)

    for subfolder, zip_filename in files.items():
        # Crear subcarpeta dentro de la carpeta principal
        subfolder_path = os.path.join(folder_path, subfolder)
        os.makedirs(subfolder_path, exist_ok=True)

        # Ruta completa del archivo .zip
        zip_path = os.path.join(base_dir, zip_filename)

        # Verificar si el archivo .zip existe
        if os.path.exists(zip_path):
            # Descomprimir el archivo directamente en la subcarpeta
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                for file in zip_ref.namelist():
                    # Comprobar si el elemento es un archivo (no un directorio)
                    if not file.endswith('/'):
                        # Extraer solo el contenido (evitando subcarpetas internas)
                        extracted_path = os.path.join(subfolder_path, os.path.basename(file))
                        with zip_ref.open(file) as source, open(extracted_path, 'wb') as target:
                            target.write(source.read())
                print(f"Descomprimido: {zip_filename} en {subfolder_path}")
        else:
            print(f"Archivo no encontrado: {zip_path}")

Descomprimido: images_train.zip en /content/train/images
Descomprimido: labels_train.zip en /content/train/labels
Descomprimido: images_val.zip en /content/val/images
Descomprimido: labels_val.zip en /content/val/labels
Descomprimido: images_test.zip en /content/test/images
Descomprimido: labels_test.zip en /content/test/labels


In [ ]:
# Crear el archivo data.yaml
data_yaml = """
train: /content/train/images
val: /content/val/images
test: /content/test/images

nc: 5  # Número de clases (ajusta según tus clases)

names:
- cells-A6sP
- lumen
- nolumen
- notumor
- tumor
"""

# Guardar el archivo
with open("/content/data.yaml", "w") as f:
    f.write(data_yaml)

print("Archivo data.yaml creado en /content/data.yaml")


Archivo data.yaml creado en /content/data.yaml


In [ ]:
import itertools
from ultralytics import YOLO
import pandas as pd
import shutil

# Definir valores probables para los hiperparámetros
param_grid = {
    'lr0': [0.001, 0.003, 0.005, 0.007, 0.01],       # Tasa de aprendizaje
    'momentum': [0.85, 0.9, 0.93, 0.95],            # Momentum
    'weight_decay': [0.0001, 0.0003, 0.0005, 0.001] # Decaimiento del peso
}

# Generar combinaciones de hiperparámetros
param_combinations = list(itertools.product(*param_grid.values()))
print(f"Número total de combinaciones: {len(param_combinations)}")

# Función para entrenar y evaluar
def train_and_evaluate(params, run_id):
    # Extraer parámetros
    lr0, momentum, weight_decay = params
    run_name = f"grid_run_{run_id}"  # Nombre único para esta ejecución

    # Cargar el modelo
    model = YOLO('yolo11n.pt')

    # Entrenar el modelo
    model.train(
        data='/content/data.yaml',
        epochs=3,                   # Épocas fijas
        imgsz=640,                   # Tamaño de imagen fijo
        batch=16,                    # Tamaño del batch fijo
        lr0=lr0,
        momentum=momentum,
        weight_decay=weight_decay,
        project='grid_search',
        name=run_name,
        verbose=False
    )

    # Evaluar el modelo
    metrics = model.val(data='/content/data.yaml', split='val')

    # Extraer mAP@50 y mAP@50-95
    results = {
        'run_id': run_id,
        'lr0': lr0,
        'momentum': momentum,
        'weight_decay': weight_decay,
        'mAP50': metrics.box.map50,
        'mAP50-95': metrics.box.map,
    }

    # Limpiar espacio (opcional)
    shutil.rmtree(f"runs/detect/{run_name}", ignore_errors=True)
    return results

# Realizar la búsqueda
results = []
for run_id, params in enumerate(param_combinations):
    print(f"Ejecutando combinación {run_id + 1}/{len(param_combinations)}: {params}")
    result = train_and_evaluate(params, run_id)
    results.append(result)

# Guardar resultados
results_df = pd.DataFrame(results)
results_df.to_csv('/content/grid_search_results.csv', index=False)

# Identificar la mejor combinación
best_result = results_df.sort_values(by='mAP50-95', ascending=False).iloc[0]
print("\nMejores hiperparámetros encontrados:")
print(best_result)

# Mostrar los hiperparámetros en un formato reutilizable
best_params = {
    'lr0': best_result['lr0'],
    'momentum': best_result['momentum'],
    'weight_decay': best_result['weight_decay']
}

print("\nPuedes usar los siguientes hiperparámetros para entrenar otro modelo:")
print(best_params)



Número total de combinaciones: 80
Ejecutando combinación 1/80: (0.001, 0.85, 0.0001)
Ultralytics 8.3.65 🚀 Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/content/data.yaml, epochs=3, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=grid_search, name=grid_run_03, exist_ok=False, pretrained=True, optimizer=auto, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save

train: Scanning /content/train/labels.cache... 5314 images, 8 backgrounds, 0 corrupt: 100%|██████████| 5322/5322 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/train/images/106_png_jpg.rf.3a611413b640f44920d80c1b6b377bcd.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/train/images/106_png_jpg.rf.41d0b720c9a46b8189277f11a37fe92d.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/train/images/106_png_jpg.rf.c0e3cc5493d949cb4e448c594c9ce229.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/train/images/106_png_jpg.rf.c3e153a450b8ee3ce952588117406445.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/train/images/135_png_jpg.rf.202abd7ae544b0e6ada1e8ec3562878f.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/train/images/135_png_jpg.rf.6fd50ecf930de39b0abc6ec857c37bad.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/train/images/155_png_jpg.rf.451c2b097d6462c535e8d7bd0b070c46.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/train/images/155_png_jpg.rf.ae0f275662b3206aa2fc70ac16064252.jpg: 1 duplicate labels removed
albumentations: Blur(p=0.01, blur_limit=


val: Scanning /content/val/labels.cache... 725 images, 1 backgrounds, 0 corrupt: 100%|██████████| 726/726 [00:00<?, ?it/s]


Plotting labels to grid_search/grid_run_03/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.001' and 'momentum=0.85' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0001), 87 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to grid_search/grid_run_03
Starting training for 3 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

        3/3      2.99G     0.9577      1.224      1.081        119        640: 100%|██████████| 333/333 [01:54<00:00,  2.91it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 23/23 [00:07<00:00,  3.06it/s]


                   all        726       5740      0.404      0.516      0.444      0.326

3 epochs completed in 0.108 hours.
Optimizer stripped from grid_search/grid_run_14/weights/last.pt, 5.5MB
Optimizer stripped from grid_search/grid_run_14/weights/best.pt, 5.5MB

Validating grid_search/grid_run_14/weights/best.pt...
Ultralytics 8.3.65 🚀 Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLO11n summary (fused): 238 layers, 2,583,127 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 23/23 [00:09<00:00,  2.38it/s]


                   all        726       5740      0.405      0.514      0.444      0.326
Speed: 0.2ms preprocess, 2.3ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to grid_search/grid_run_14
Ultralytics 8.3.65 🚀 Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLO11n summary (fused): 238 layers, 2,583,127 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning /content/val/labels.cache... 725 images, 1 backgrounds, 0 corrupt: 100%|██████████| 726/726 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   7%|▋         | 3/46 [00:02<00:35,  1.23it/s]